In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('zeroshot')

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': True, 'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters, 'refit_full': True,
                'set_best_to_refit_full': True, 'save_bag_folds': True, 'num_bag_folds': 5}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(train_data, presets=custom_preset, excluded_model_types=['KNN', 'CAT', 'GBM', 'NN_TORCH'])  

No path specified. Models will be saved in: "AutogluonModels\ag-20240824_021558"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       24.22 GB / 31.75 GB (76.3%)
Disk Space Avail:   773.91 GB / 952.44 GB (81.3%)
Presets specified: [{'auto_stack': True, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': [{}, {'activation': 'elu', 'dropout_prob': 0.10077639529843717, 'hidden_size': 108, 'learning_rate': 0.002735937344002146, 'num_layers': 4, 'use_batchnorm': True, 'weight_decay': 1.356433327634438e-12, 'ag_args': {'name_suffix': '_r79', 'priority': -2}}, {'activation': 'elu', 'dropout_prob': 0.11897478034205347, 'hidden_size': 213, 'learning_rate': 0.0010474382260641949, 'num_layers': 4, 'use_batchnorm': False, 'weight_decay': 5.594471067786272e-10, 'ag_args': {'name_suf

In [ ]:
# %%

metrics = ['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test', 'fit_time']

df_leaders = predictor.leaderboard(test_data)

df_leaders.head(40)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L3,-0.120198,-0.117657,log_loss,49.370204,1383.488130
1,WeightedEnsemble_L3_FULL,-0.125161,NaN,log_loss,12.208002,677.934532
2,NeuralNetFastAI_r134_BAG_L2,-0.125275,-0.132050,log_loss,43.378534,696.523790
3,NeuralNetFastAI_r69_BAG_L2,-0.125503,-0.138941,log_loss,44.002533,690.662784
4,NeuralNetFastAI_r145_BAG_L2,-0.125706,-0.139896,log_loss,44.183533,712.214793
5,NeuralNetFastAI_BAG_L2,-0.126115,-0.139391,log_loss,43.713534,684.937783
6,NeuralNetFastAI_r156_BAG_L2,-0.126162,-0.129765,log_loss,43.474534,675.203783
7,NeuralNetFastAI_r187_BAG_L2,-0.126235,-0.133918,log_loss,43.436533,679.111784
8,NeuralNetFastAI_r127_BAG_L2,-0.126461,-0.131760,log_loss,43.308534,671.605782
9,NeuralNetFastAI_r102_BAG_L2,-0.127288,-0.132795,log_loss,43.268529,670.405780


In [ ]:
# %%

df_leaders.tail(20)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
204,NeuralNetFastAI_r100_BAG_L1_FULL,-0.387363,NaN,log_loss,0.040000,4.373003
205,NeuralNetFastAI_r88_BAG_L1_FULL,-0.402717,NaN,log_loss,0.061000,3.255000
206,LinearModelLasso_BAG_L1_FULL,-0.402998,NaN,log_loss,0.022000,3.849610
207,LinearModelRidge_BAG_L1_FULL,-0.404254,NaN,log_loss,0.018000,3.813901
208,LinearModelElasticNet_BAG_L1_FULL,-0.404254,NaN,log_loss,0.019001,3.841088
209,LinearModelBase_BAG_L1_FULL,-0.412393,NaN,log_loss,0.029000,4.716308
210,LinearModelBase_BAG_L1,-0.421172,-0.412298,log_loss,0.070000,5.412909
211,NeuralNetFastAI_r103_BAG_L1_FULL,-0.429689,NaN,log_loss,0.090999,14.261923
212,NeuralNetFastAI_r65_BAG_L1_FULL,-0.449745,NaN,log_loss,0.085998,4.055966
213,NeuralNetFastAI_r143_BAG_L2_FULL,-0.456934,NaN,log_loss,10.557003,405.547013


In [ ]:
# %%

y_pred = predictor.predict_proba(test_data.drop(columns=[label]))
y_pred.head()

# predictor.evaluate(test_data, silent=True)

,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
0,0.000017,0.000721,0.000013,0.007177,0.988115,0.000986,0.000012,0.001390,0.001495,0.000010,0.000018,0.000015,0.000032
1,0.000250,0.000340,0.000311,0.000202,0.001451,0.371085,0.621231,0.003786,0.000158,0.000185,0.000421,0.000328,0.000252
2,0.000016,0.000018,0.000019,0.000009,0.000012,0.000336,0.999147,0.000365,0.000011,0.000011,0.000017,0.000018,0.000020
3,0.000031,0.000131,0.000028,0.000018,0.000013,0.000018,0.000016,0.003599,0.983552,0.011642,0.000112,0.000811,0.000029
4,0.000026,0.000024,0.000021,0.000015,0.000012,0.000012,0.005904,0.992961,0.000938,0.000018,0.000021,0.000028,0.000020


In [ ]:
# %%

# "AutogluonModels\ag-20240821_072221"